## Алгоритм Дейкстры

In [1]:
from math import inf

with open('input.txt', 'r') as f:
    N, S, F = list(map(int, f.readline().strip().split()))
    adjacency_matrix = [[inf] * (N + 1)]
    for i in range(N):
        row = list(map(int, f.readline().strip().split()))
        row = [i if i != -1 else inf for i in row]
        adjacency_matrix.append([inf] + row)

def dijkstra(adjacency_matrix, S):
    N = len(adjacency_matrix) - 1
    distances = [inf] * (N + 1)
    visited = [False] * (N + 1)
    way_back = [-1] * (N + 1)
    def argmin(a):
        min_elem = None
        for i, elem in enumerate(visited):
            if elem == False:
                min_elem = a[i]
                break
        ind = 0
        for i, etre in enumerate(a):
            if visited[i] == False:
                if etre < min_elem:
                    min_elem = etre
                    ind = i
        return ind, min_elem
    distances[S] = 0
    for vertice in range(N):
        next_vertice, part_dist = argmin(distances)
        visited[next_vertice] = True
        voisins = [i for i in range(N + 1) if adjacency_matrix[next_vertice][i] != inf]
        for etre in voisins:
            new_dist = part_dist + adjacency_matrix[next_vertice][etre]
            if new_dist < distances[etre]:
                distances[etre] = new_dist
                way_back[etre] = next_vertice
    return distances, way_back

distances, way_back = dijkstra(adjacency_matrix, S)

if distances[F] == inf:
    print(-1)
else:
    drawing = []
    current_vertice = F
    while current_vertice != -1:
        drawing.append(current_vertice)
        current_vertice = way_back[current_vertice]
    print(*drawing[::-1])

2 3 1


## Быстрый алгоритм Дейкстры

In [2]:
inf = 10 ** 9
from collections import defaultdict
import heapq

class HeapWithRemoval():
    def __init__(self, n):
        self.heap = [(0, i, 1440) for i in range(n + 1)]
        self.deleted = defaultdict(int)
    def add(self, value):
        if self.deleted[value]:
            self.deleted[value] -= 1
        else:
            heapq.heappush(self.heap, value)
    def remove(self, value):
        self.deleted[value] += 1
        while self.heap and self.deleted[self.heap[0]]:
            self.deleted[self.heap[0]] -= 1
            heapq.heappop(self.heap)
    def minimum(self):
        return self.heap[0] if self.heap else None
    
def dijkstra(adjacency_list, N, S):
    distances = HeapWithRemoval(N)
    final_distances = [0] * (N + 1)
    final_times = [1440] * (N + 1)
    final_times[S] = 0
    final_distances[S] = 10 ** 9
    distances.remove((0, S, 1440))
    distances.add((-10 ** 9, S, 0))
    for vertice in range(N):
        part_dist, next_vertice, add_time = distances.minimum()
        distances.remove((part_dist, next_vertice, add_time))
        voisins = adjacency_list[next_vertice]
        for i, etre in enumerate(voisins):
            new_dist = min(-part_dist, adjacency_list[next_vertice][i][0])
            new_time = add_time + adjacency_list[next_vertice][i][2]
            if new_dist > final_distances[etre[1]]:
                if new_time <= 1440:
                    distances.remove((-final_distances[etre[1]], etre[1], final_times[etre[1]]))
                    distances.add((-new_dist, etre[1], new_time))
                    final_distances[etre[1]] = new_dist
                    final_times[etre[1]] = new_time
    return final_distances

with open('input.txt', 'r') as f:
    N, K = list(map(int, f.readline().strip().split()))
    adjacency_list = defaultdict(list)
    for i in range(K):
        a, b, t, l = list(map(int, f.readline().strip().split()))
        adjacency_list[a].append((l, b, t))#!!!!!!!!!!!!!!!Сначала расстояния
        adjacency_list[b].append((l, a, t))
    
final_distances = dijkstra(adjacency_list, N, 1)

print(max(0, final_distances[N] - 3000000) // 100)

2


## Задача D - автобусы в Васюках

In [24]:
from math import inf
with open('input.txt', 'r') as f:
    n = int(f.readline().strip())
    min_time = [inf] * (n + 1)
    visited = [False] * (n + 1)
    adjacency_matrix = [[[] for i in range(n + 1)] for j in range(n + 1)]
    d, v = list(map(int, f.readline().strip().split()))
    r = int(f.readline().strip())
    data = []
    for i in range(r):
        vil_from, time_from, vil_to, time_to = list(map(int, f.readline().strip().split()))
        data.append((vil_from, time_from, vil_to, time_to))
    data = sorted(data, key=lambda x:x[3])
    for ever in data:
        adjacency_matrix[ever[0]][ever[2]].append((ever[1], ever[3]))

def argmin(a):
    min_elem = None
    for i, elem in enumerate(visited):
        if elem == False:
            min_elem = a[i]
            break
    ind = 0
    for i, etre in enumerate(a):
        if visited[i] == False:
            if etre < min_elem:
                min_elem = etre
                ind = i
    return ind, min_elem

min_time[d] = 0
for vertice in range(n):
    next_vertice, part_time = argmin(min_time)
    visited[next_vertice] = True
    voisins = [i for i in range(n + 1) if len(adjacency_matrix[next_vertice][i])]
    for etre in voisins:
        depature_time, arrival_time = None, None
        for ever in adjacency_matrix[next_vertice][etre]:
            if ever[0] >= part_time:
                depature_time = ever[0]
                arrival_time = ever[1]
                break
        if depature_time is not None:
            if arrival_time < min_time[etre]:
                min_time[etre] = arrival_time
if min_time[v] == inf:
    print(-1)
else:
    print(min_time[v])

29


## Задача E - на санях

In [1]:
from math import inf
from collections import defaultdict
import heapq

class HeapWithRemoval():
    def __init__(self, n):
        self.heap = [(inf, i) for i in range(n + 1)]
        self.deleted = defaultdict(int)
    def add(self, value):
        if self.deleted[value]:
            self.deleted[value] -= 1
        else:
            heapq.heappush(self.heap, value)
    def remove(self, value):
        self.deleted[value] += 1
        while self.heap and self.deleted[self.heap[0]]:
            self.deleted[self.heap[0]] -= 1
            heapq.heappop(self.heap)
    def minimum(self):
        return self.heap[0] if self.heap else None
        
cities = [None]
with open('input.txt', 'r') as f:
    N = int(f.readline().strip())
    for i in range(N):
        T, V = list(map(int, f.readline().strip().split()))
        cities.append((T, V))
    adjacency_list = defaultdict(list)
    for i in range(N - 1):
        a, b, l = list(map(int, f.readline().strip().split()))
        adjacency_list[a].append((l, b))
        adjacency_list[b].append((l, a))

def DFS(next_vertice, next_distance):
    visited[next_vertice] = True
    voisins = adjacency_list[next_vertice]
    for i, vertice in enumerate(voisins):
        if not visited[vertice[1]]:
            next_distance += adjacency_list[next_vertice][i][0]
            distances[vertice[1]] = next_distance
            DFS(vertice[1], next_distance)
            next_distance -= adjacency_list[next_vertice][i][0]
    
time_matrix = [[inf] * (N + 1)]
for etre in range(1, N + 1):
    distances = [inf] * (N + 1)
    visited = [False] * (N + 1)
    final_times = [inf] * (N + 1)
    distances[etre] = 0
    DFS(etre, 0)
    for q, dist in enumerate(distances[1:], 1):
        if dist:
            delay = cities[etre][0]
            final_time = delay + dist / cities[etre][1]
        else:
            final_time = 0
            
        final_times[q] = final_time
    time_matrix.append(final_times)

way_back = [-1] * (N + 1)
maximal_time = 0
max_ind = 0

distances = HeapWithRemoval(N)
final_distances = [inf] * (N + 1)
final_distances[1] = 0
distances.remove((inf, 1))
distances.add((0, 1))
for vertice in range(N):
    part_dist, next_vertice = distances.minimum()
    distances.remove((part_dist, next_vertice))
    for ind in range(1, N + 1):
        new_dist = part_dist + time_matrix[ind][next_vertice]
        if new_dist < final_distances[ind]:
            distances.remove((final_distances[ind], ind))
            distances.add((new_dist, ind))
            final_distances[ind] = new_dist
            way_back[ind] = next_vertice
for i in range(2, N + 1):
    if final_distances[i] >= maximal_time:
        maximal_time = final_distances[i]
        max_ind = i
print(maximal_time)
drawing = []
current_vertice = max_ind
while current_vertice != -1:
    drawing.append(current_vertice)
    current_vertice = way_back[current_vertice]
print(*drawing)

276178.9580825514
1103 22 16 10 3 1
